In [10]:
import pymongo

def create_connection():
    try:
        client = pymongo.MongoClient("mongodb://localhost:27017/")
        return client["IUM-Calcio"], client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None, None


def close_connection(client):
    # Close connection
    if client is not None:  # Check for None explicitly
        client.close()
        
def retrieve(db):
    try:
        collection = db["clubs"]
        query_clubs = {"average_age": {"$gt": 25}, "stadium_seats": {"$gt": 80000}}
        projection = {"name": 1}
        old_players_clubs = list(collection.find(query_clubs, projection))
        return old_players_clubs
    except Exception as e:
        print(f"Retrieval of Clubs failed: {e}")
        return []
    
db, client = create_connection()
